In [78]:
import requests
import io
import json
import csv
from requests import Response
from requests.exceptions import HTTPError,ConnectionError,Timeout
from pydantic import BaseModel,RootModel,Field,field_validator,field_serializer
from datetime import datetime
from pprint import pprint

try:
    url="https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"
    response=requests.get(url)
    response.raise_for_status()
    content=response.text
except ConnectionError:
    print("連線伺服器發生錯誤")
except Timeout:
    print("伺服器太忙,沒有回應")
except HTTPError:
    print("伺服器回應連線錯誤")
except Exception:
    print("不知名的錯誤")



file=io.StringIO(content)
content=json.load(file)

class Station():
    def __init__(self,name:str,no:float):
        self.name = name
        self.no = no

for station in content:
    no:str=station['sno']
    name:str=station['sna']

    class Station(BaseModel):
        站名:str=Field(alias="sna")
        行政區:str=Field(alias="sarea")
        日期:datetime=Field(alias="mday")
        地址:str=Field(alias="ar")
        狀態:str=Field(alias="act")
        總車輛數:int=Field(alias="total")
        可借:int=Field(alias="available_rent_bikes")
        可還:int=Field(alias="available_return_bikes")

    @field_validator("站名")
    @classmethod
    def slice_str(cls,name:str):
        return name[11:]
    
    @field_validator("狀態",mode="before")
    @classmethod
    def slice_str(cls,status:str):
        if status=="1":
            return "營業中"
        else:
            return "維修中"
        
    @field_serializer("日期")
    def date_serializer(self,date:datetime)->str:
        return date.strftime(f"中華民國{date.year-1911}年%m月%d日 %H時%M分%S秒")
    
    if no=="500108150":
        print(no,name)
    if no=="500108047":
        print(no,name)

500108047 YouBike2.0_瑞光公共住宅(瑞光路)
500108150 YouBike2.0_台灣世曦大樓
